In [1]:
import pandas as pd
import numpy as np
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import *
import re
from sklearn.preprocessing import LabelEncoder
from sklearn.feature_extraction.text import TfidfVectorizer

In [2]:
data = pd.read_csv("C:\\Users\\manideepreddy\\OneDrive\\Desktop\\fake news.csv")

In [3]:
data.head()

,Unnamed: 0,id,title,author,text,label
0,0,0,House Dem Aide: We Didn’t Even See Comey’s Let...,Darrell Lucus,House Dem Aide: We Didn’t Even See Comey’s Let...,1
1,1,1,"FLYNN: Hillary Clinton, Big Woman on Campus - ...",Daniel J. Flynn,Ever get the feeling your life circles the rou...,0
2,2,2,Why the Truth Might Get You Fired,Consortiumnews.com,"Why the Truth Might Get You Fired October 29, ...",1
3,3,3,15 Civilians Killed In Single US Airstrike Hav...,Jessica Purkiss,Videos 15 Civilians Killed In Single US Airstr...,1
4,4,4,Iranian woman jailed for fictional unpublished...,Howard Portnoy,Print \nAn Iranian woman has been sentenced to...,1


In [4]:
data = data.drop("Unnamed: 0",axis = 1)

In [5]:
data.head()

,id,title,author,text,label
0,0,House Dem Aide: We Didn’t Even See Comey’s Let...,Darrell Lucus,House Dem Aide: We Didn’t Even See Comey’s Let...,1
1,1,"FLYNN: Hillary Clinton, Big Woman on Campus - ...",Daniel J. Flynn,Ever get the feeling your life circles the rou...,0
2,2,Why the Truth Might Get You Fired,Consortiumnews.com,"Why the Truth Might Get You Fired October 29, ...",1
3,3,15 Civilians Killed In Single US Airstrike Hav...,Jessica Purkiss,Videos 15 Civilians Killed In Single US Airstr...,1
4,4,Iranian woman jailed for fictional unpublished...,Howard Portnoy,Print \nAn Iranian woman has been sentenced to...,1


In [6]:
data.shape

(18285, 5)

In [7]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 18285 entries, 0 to 18284
Data columns (total 5 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   id      18285 non-null  int64 
 1   title   18285 non-null  object
 2   author  18285 non-null  object
 3   text    18285 non-null  object
 4   label   18285 non-null  int64 
dtypes: int64(2), object(3)
memory usage: 714.4+ KB


In [8]:
data['content'] = data['author']+' '+data['title']

In [9]:
data.isnull().sum()

id         0
title      0
author     0
text       0
label      0
content    0
dtype: int64

In [10]:
data.head()

,id,title,author,text,label,content
0,0,House Dem Aide: We Didn’t Even See Comey’s Let...,Darrell Lucus,House Dem Aide: We Didn’t Even See Comey’s Let...,1,Darrell Lucus House Dem Aide: We Didn’t Even S...
1,1,"FLYNN: Hillary Clinton, Big Woman on Campus - ...",Daniel J. Flynn,Ever get the feeling your life circles the rou...,0,"Daniel J. Flynn FLYNN: Hillary Clinton, Big Wo..."
2,2,Why the Truth Might Get You Fired,Consortiumnews.com,"Why the Truth Might Get You Fired October 29, ...",1,Consortiumnews.com Why the Truth Might Get You...
3,3,15 Civilians Killed In Single US Airstrike Hav...,Jessica Purkiss,Videos 15 Civilians Killed In Single US Airstr...,1,Jessica Purkiss 15 Civilians Killed In Single ...
4,4,Iranian woman jailed for fictional unpublished...,Howard Portnoy,Print \nAn Iranian woman has been sentenced to...,1,Howard Portnoy Iranian woman jailed for fictio...


In [11]:
data["content"]

0        Darrell Lucus House Dem Aide: We Didn’t Even S...
1        Daniel J. Flynn FLYNN: Hillary Clinton, Big Wo...
2        Consortiumnews.com Why the Truth Might Get You...
3        Jessica Purkiss 15 Civilians Killed In Single ...
4        Howard Portnoy Iranian woman jailed for fictio...
                               ...                        
18280    Jerome Hudson Rapper T.I.: Trump a ’Poster Chi...
18281    Benjamin Hoffman N.F.L. Playoffs: Schedule, Ma...
18282    Michael J. de la Merced and Rachel Abrams Macy...
18283    Alex Ansary NATO, Russia To Hold Parallel Exer...
18284              David Swanson What Keeps the F-35 Alive
Name: content, Length: 18285, dtype: object

In [12]:
import nltk
nltk.download("stopwords")

[nltk_data] Error loading stopwords: [WinError 10053] An established
[nltk_data]     connection was aborted by the software in your host
[nltk_data]     machine


False

In [13]:
print(stopwords.words("english"))

['i', 'me', 'my', 'myself', 'we', 'our', 'ours', 'ourselves', 'you', "you're", "you've", "you'll", "you'd", 'your', 'yours', 'yourself', 'yourselves', 'he', 'him', 'his', 'himself', 'she', "she's", 'her', 'hers', 'herself', 'it', "it's", 'its', 'itself', 'they', 'them', 'their', 'theirs', 'themselves', 'what', 'which', 'who', 'whom', 'this', 'that', "that'll", 'these', 'those', 'am', 'is', 'are', 'was', 'were', 'be', 'been', 'being', 'have', 'has', 'had', 'having', 'do', 'does', 'did', 'doing', 'a', 'an', 'the', 'and', 'but', 'if', 'or', 'because', 'as', 'until', 'while', 'of', 'at', 'by', 'for', 'with', 'about', 'against', 'between', 'into', 'through', 'during', 'before', 'after', 'above', 'below', 'to', 'from', 'up', 'down', 'in', 'out', 'on', 'off', 'over', 'under', 'again', 'further', 'then', 'once', 'here', 'there', 'when', 'where', 'why', 'how', 'all', 'any', 'both', 'each', 'few', 'more', 'most', 'other', 'some', 'such', 'no', 'nor', 'not', 'only', 'own', 'same', 'so', 'than', '

In [14]:
portstem = PorterStemmer()

In [15]:
def stemming(content):
    stemmed_content = re.sub("[^a-zA-Z]"," ",content)
    stemmed_content = stemmed_content.lower()
    stemmed_content = stemmed_content.split()
    stemmed_content = [portstem.stem(word) for word in stemmed_content if not word in stopwords.words("english")]
    stemmed_content = ' '.join(stemmed_content)
    return stemmed_content

In [16]:
data["content"] = data["content"].apply(stemming)

In [17]:
data["content"]

0        darrel lucu hous dem aid even see comey letter...
1        daniel j flynn flynn hillari clinton big woman...
2                   consortiumnew com truth might get fire
3        jessica purkiss civilian kill singl us airstri...
4        howard portnoy iranian woman jail fiction unpu...
                               ...                        
18280    jerom hudson rapper trump poster child white s...
18281    benjamin hoffman n f l playoff schedul matchup...
18282    michael j de la merc rachel abram maci said re...
18283    alex ansari nato russia hold parallel exercis ...
18284                            david swanson keep f aliv
Name: content, Length: 18285, dtype: object

In [18]:
x =data['content'].values
y = data['label'].values

In [19]:
vectorizer = TfidfVectorizer()

In [20]:
x = vectorizer.fit_transform(x)

In [21]:
print(x)

  (0, 14626)	0.2853880981846006
  (0, 2312)	0.3745612250433202
  (0, 7190)	0.24556189342497173
  (0, 8048)	0.29347549279156676
  (0, 2757)	0.2466340295002162
  (0, 12567)	0.25566372256502734
  (0, 4637)	0.23016077319140021
  (0, 247)	0.26982554594264346
  (0, 3543)	0.2684494960336511
  (0, 6552)	0.21745594418933306
  (0, 8310)	0.3609049070394367
  (0, 3359)	0.3609049070394367
  (1, 1764)	0.15099851642776987
  (1, 2066)	0.3819189043603919
  (1, 15663)	0.30530279633389806
  (1, 1391)	0.2961798071396214
  (1, 2619)	0.1936832753563371
  (1, 6377)	0.19285723710368194
  (1, 5140)	0.7119376870709987
  (1, 3328)	0.2623789770430963
  (2, 5031)	0.38709995799949964
  (2, 5579)	0.3490632212946542
  (2, 8973)	0.4948460479407663
  (2, 14560)	0.4180284001448272
  (2, 2743)	0.31437590974242646
  :	:
  (18282, 1194)	0.33325787344622426
  (18282, 12239)	0.2527439079680461
  (18282, 11515)	0.2748252773264482
  (18282, 13966)	0.30927367322621613
  (18282, 11321)	0.24588400571511218
  (18282, 8879)	0.29296

In [22]:
x_train,x_test,y_train,y_test = train_test_split(x,y,test_size = 0.2,random_state = 101)

In [23]:
print(x_train.shape,x_test.shape,y_train.shape,y_test.shape)

(14628, 15960) (3657, 15960) (14628,) (3657,)


In [24]:
log = LogisticRegression()

In [25]:
log

LogisticRegression()

In [26]:
log.fit(x_train,y_train)

LogisticRegression()

In [27]:
p = log.predict(x_test)

In [28]:
p

array([1, 0, 0, ..., 1, 0, 0], dtype=int64)

In [29]:
confusion_matrix(p,y_test)

array([[1987,   18],
       [  39, 1613]], dtype=int64)

In [30]:
accuracy_score(p,y_test)

0.9844134536505332